In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import sequence
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Masking
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import gc
import os
import time
from collections import Counter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import sequence
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Masking,concatenate,Input
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import gc
import os
import time
from collections import Counter

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import sequence
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Masking
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import gc
import os
import time
from collections import Counter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import sequence
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Masking,concatenate,Input
from keras.callbacks import ModelCheckpoint

import tsfresh
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import gc
import os
import time
from collections import Counter

def mywloss(y_true,y_pred):
    yc=tf.clip_by_value(y_pred,1e-15,1-1e-15)
    loss=-(tf.reduce_mean(tf.reduce_mean(y_true*tf.log(yc),axis=0)/wtable))
    return loss

def multi_weighted_logloss(y_ohe, y_p):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1-1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos
    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss

def normalize(df):
    df_mean = df.mean(axis=0)
    df.fillna(df_mean, inplace=True)

    df_new = df.copy()
    ss = StandardScaler()
    return ss.fit_transform(df_new)

def plot_loss_acc(history):
    plt.plot(history.history['loss'][1:])
    plt.plot(history.history['val_loss'][1:])
    plt.title('model loss')
    plt.ylabel('val_loss')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['acc'][1:])
    plt.plot(history.history['val_acc'][1:])
    plt.title('model Accuracy')
    plt.ylabel('val_acc')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')
    plt.show()

In [ ]:


gc.enable()
train = pd.read_csv('../input/training_set.csv')
meta_train = pd.read_csv('../input/training_set_metadata.csv')
meta_train.head()

full_train = train.reset_index().merge(
    right=meta_train,
    how='outer',
    on='object_id'
)

del train
gc.collect()

splits = full_train[['object_id']].diff()["object_id"].nonzero()[0]
oof_df = full_train[['object_id']].iloc[splits[1:]-1]
passband = full_train[["passband"]]
passband = to_categorical(passband)
meta_train = full_train[meta_train.keys()].iloc[splits[1:]-1]
del meta_train['object_id'], meta_train['target']

if 'target' in full_train:
    y = full_train['target'][splits[1:] - 1]
    del full_train['target']
classes = sorted(y.unique())
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

if 'object_id' in full_train:
    del full_train['passband'], full_train['object_id'], full_train['distmod'], full_train['hostgal_specz'], full_train['hostgal_photoz'], full_train['hostgal_photoz_err'], full_train['mwebv']
    del full_train['ra'], full_train['decl'], full_train['gal_l'],full_train['gal_b'],full_train['ddf']

full_train_ss = normalize(full_train)
train_data2 = normalize(meta_train)
sequence_length = max(np.diff(splits))
full_train_ss = np.concatenate([full_train_ss, passband], axis=1)
train_data = np.split(full_train_ss, splits[1:-1])
train_data = sequence.pad_sequences(train_data, maxlen=sequence_length, dtype=train_data[0].dtype)

unique_y = np.unique(y)
class_map = dict()
for i,val in enumerate(unique_y):
    class_map[val] = i

y_map = np.zeros((y.shape[0],))
y_map = np.array([class_map[val] for val in y])
y_categorical = to_categorical(y_map)

y_count = Counter(y_map)
wtable = np.zeros((len(unique_y),))
for i in range(len(unique_y)):
    wtable[i] = y_count[i]/y_map.shape[0]


In [ ]:
K.clear_session()

def build_model(dropout_rate=.2, activation="relu"):
    input1 = Input(shape=(train_data.shape[-2], train_data.shape[-1]))
    x1 = Masking(mask_value=0.)(input1)
    x1 = LSTM(50, return_sequences=True)(x1)
    x1 = LSTM(25)(x1)
    x1 = Dense(30, activation=activation)(x1)
    x1 = Dropout(dropout_rate)(x1)

    input2 = Input(shape=(train_data2.shape[-1],))
    x2 = Dense(30, activation=activation)(input2)
    x2 = Dropout(dropout_rate)(x2)
    x2 = Dense(30, activation=activation)(x2)
    x2 = Dropout(dropout_rate)(x2)

    x3 = concatenate([x1, x2])
    x3 = Dense(30, activation=activation)(x3)
    x3 = Dropout(dropout_rate)(x3)
    x3 = Dense(30, activation=activation)(x3)
    x3 = Dropout(dropout_rate)(x3)
    x3 = Dense(len(classes), activation='softmax')(x3)

    return Model(inputs=[input1, input2], output=x3)


In [ ]:
K.clear_session()
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clfs = []
oof_preds = np.zeros((len(oof_df), len(classes)))
epochs = 600
batch_size = 10000
start_time = time.time()
for fold_, (trn_, val_) in enumerate(folds.split(y_map, y_map)):
    checkPoint = ModelCheckpoint("./keras.model",monitor='val_loss',mode = 'min', save_best_only=True, verbose=0)
    x_train, x_train2, y_train = train_data[trn_], train_data2[trn_], y_categorical[trn_]
    x_valid, x_valid2, y_valid = train_data[val_], train_data2[val_], y_categorical[val_]

    model = build_model(dropout_rate=0.2,activation='relu')
    model.compile(loss=mywloss, optimizer='adam', metrics=['accuracy'])
    history = model.fit([x_train, x_train2], y_train,
                    validation_data=[[x_valid, x_valid2], y_valid],
                    epochs=epochs,
                    batch_size=batch_size,shuffle=True,verbose=0,callbacks=[checkPoint])

    plot_loss_acc(history)

    print('Loading Best Model')
    model.load_weights('./keras.model')
    # # Get predicted probabilities for each class
    oof_preds[val_, :] = model.predict([x_valid,x_valid2],batch_size=batch_size)
    print(multi_weighted_logloss(y_valid, model.predict([x_valid,x_valid2],batch_size=batch_size)))
    clfs.append(model)
    break

end_time = time.time()
print("Took %.5f minutes" % ((end_time - start_time) / 60.0))
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_categorical,oof_preds))


In [ ]:
import tsfresh
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
#gc.enable()
train = pd.read_csv('../input/training_set.csv')
meta_train = pd.read_csv('../input/training_set_metadata.csv')
meta_train.head()

full_train = train.reset_index().merge(
    right=meta_train,
    how='outer',
    on='object_id'
)

#del train
#gc.collect()


In [ ]:
enc = OneHotEncoder()
meta_train['target'] = meta_train['target'].astype("object")
enc.fit(meta_train[["target"]])
target = meta_train[["object_id", "target"]]
target.index = target['object_id']
new_features = None
for i in [0,1,2,3,4,5]:
    passband_train = train[train['passband'] == i]
    features = tsfresh.extract_relevant_features(
        passband_train,
        target.loc[passband_train["object_id"].unique()]["target"],
        column_id="object_id", 
        column_sort="mjd",
        column_value="flux"
    )

    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(features, target.loc[features.index][["target"]].astype("str"))

    model = SelectFromModel(clf, prefit=True, max_features=20)
    features = features[features.columns[model.get_support()]]
    if new_features is not None:
        new_features = pd.concat([new_features, features], axis=1)
    else:
        new_features = features

In [ ]:
new_features.to_csv("new_features.csv")

In [ ]:
new_features.shape


In [ ]:
def multi_weighted_logloss(y_true, y_preds, classes, class_weights):
    """
    refactor from
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weights[k] for k in sorted(class_weights.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgbm_multi_weighted_logloss(y_true, y_preds):
    """
    refactor from
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """  
    # Taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weights = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}

    loss = multi_weighted_logloss(y_true, y_preds, classes, class_weights)
    return 'wloss', loss, False


def xgb_multi_weighted_logloss(y_predicted, y_true, classes, class_weights):
    loss = multi_weighted_logloss(y_true.get_label(), y_predicted, 
                                  classes, class_weights)
    return 'wloss', loss


def save_importances(importances_):
    mean_gain = importances_[['gain', 'feature']].groupby('feature').mean()
    importances_['mean_gain'] = importances_['feature'].map(mean_gain['gain'])
    return importances_


def xgb_modeling_cross_validation(params,
                                  full_train, 
                                  y, 
                                  classes, 
                                  class_weights, 
                                  nr_fold=5, 
                                  random_state=1):
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}

    # loss function
    func_loss = partial(xgb_multi_weighted_logloss, 
                        classes=classes, 
                        class_weights=class_weights)

    clfs = []
    importances = pd.DataFrame()
    folds = StratifiedKFold(n_splits=nr_fold, 
                            shuffle=True, 
                            random_state=random_state)
    
    oof_preds = np.zeros((len(full_train), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = full_train.iloc[trn_], y.iloc[trn_]
        val_x, val_y = full_train.iloc[val_], y.iloc[val_]
    
        clf = XGBClassifier(**params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=func_loss,
            verbose=100,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        clfs.append(clf)

        oof_preds[val_, :] = clf.predict_proba(val_x, ntree_limit=clf.best_ntree_limit)
        print('no {}-fold loss: {}'.format(fold_ + 1, 
              multi_weighted_logloss(val_y, oof_preds[val_, :], 
                                     classes, class_weights)))
    
        imp_df = pd.DataFrame({
                'feature': full_train.columns,
                'gain': clf.feature_importances_,
                'fold': [fold_ + 1] * len(full_train.columns),
                })
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

    score = multi_weighted_logloss(y_true=y, y_preds=oof_preds, 
                                   classes=classes, class_weights=class_weights)
    print('MULTI WEIGHTED LOG LOSS: {:.5f}'.format(score))
    df_importances = save_importances(importances_=importances)
    df_importances.to_csv('xgb_importances.csv', index=False)
    
    return clfs, score


def lgbm_modeling_cross_validation(params,
                                   full_train, 
                                   y, 
                                   classes, 
                                   class_weights, 
                                   nr_fold=5, 
                                   random_state=1):

    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}

    clfs = []
    importances = pd.DataFrame()
    folds = StratifiedKFold(n_splits=nr_fold, 
                            shuffle=True, 
                            random_state=random_state)
    
    oof_preds = np.zeros((len(full_train), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = full_train.iloc[trn_], y.iloc[trn_]
        val_x, val_y = full_train.iloc[val_], y.iloc[val_]
    
        clf = LGBMClassifier(**params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgbm_multi_weighted_logloss,
            verbose=100,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        clfs.append(clf)

        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        print('no {}-fold loss: {}'.format(fold_ + 1, 
              multi_weighted_logloss(val_y, oof_preds[val_, :], 
                                     classes, class_weights)))
    
        imp_df = pd.DataFrame({
                'feature': full_train.columns,
                'gain': clf.feature_importances_,
                'fold': [fold_ + 1] * len(full_train.columns),
                })
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

    score = multi_weighted_logloss(y_true=y, y_preds=oof_preds, 
                                   classes=classes, class_weights=class_weights)
    print('MULTI WEIGHTED LOG LOSS: {:.5f}'.format(score))
    df_importances = save_importances(importances_=importances)
    df_importances.to_csv('lgbm_importances.csv', index=False)
    
    return clfs, score

In [ ]:
   best_params = {
            'device': 'cpu', 
            'objective': 'multiclass', 
            'num_class': 14, 
            'boosting_type': 'gbdt', 
            'n_jobs': -1, 
            'max_depth': 7, 
            'n_estimators': 500, 
            'subsample_freq': 2, 
            'subsample_for_bin': 5000, 
            'min_data_per_group': 100, 
            'max_cat_to_onehot': 4, 
            'cat_l2': 1.0, 
            'cat_smooth': 59.5, 
            'max_cat_threshold': 32, 
            'metric_freq': 10, 
            'verbosity': -1, 
            'metric': 'multi_logloss', 
            'xgboost_dart_mode': False, 
            'uniform_drop': False, 
            'colsample_bytree': 0.5, 
            'drop_rate': 0.173, 
            'learning_rate': 0.0267, 
            'max_drop': 5, 
            'min_child_samples': 10, 
            'min_child_weight': 100.0, 
            'min_split_gain': 0.1, 
            'num_leaves': 7, 
            'reg_alpha': 0.1, 
            'reg_lambda': 0.00023, 
            'skip_drop': 0.44, 
            'subsample': 0.75}


In [ ]:
from lightgbm import LGBMClassifier
classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
class_weights = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}

full_features = pd.merge(new_features, meta_train, left_on=new_features.index, right_on="object_id")
full_features = full_features.drop("target", axis=1)
lgbm_modeling_cross_validation(best_params,
                                   full_features, 
                                   target["target"].astype("str"), 
                                   classes, 
                                   class_weights, 
                                   nr_fold=5, 
                                   random_state=1)

In [ ]:
y = target["target"].astype("str")
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}

clf = LGBMClassifier(**best_params)
clf.fit(
    full_features, y,
    #eval_set=[(trn_x, trn_y), (val_x, val_y)],
    eval_metric=lgbm_multi_weighted_logloss,
    verbose=100,
    #early_stopping_rounds=50,
    sample_weight=y.map(weights)
)


In [ ]:
meta_test = pd.read_csv("../input/PLAsTiCC-2018/test_set_metadata.csv")
try:
    os.remove('predictions.csv')
except FileNotFoundError:
    pass
chunks = 5000000
for i_c, test in enumerate(pd.read_csv("../input/PLAsTiCC-2018/test_set.csv", chunksize=chunks, iterator=True)):
    test_features = None
    for i in [0,1,2,3,4,5]:
        x = tsfresh.feature_extraction.settings.from_columns(new_features.columns[i*20+1:(i+1)*20+1])
        passband_test = test[test["passband"] == i]
        t_features = tsfresh.extract_features(
                passband_test,
                #test_target.loc[passband_train["object_id"].unique()]["target"],
                column_id="object_id", 
                column_sort="mjd",
                column_value="flux",
                kind_to_fc_parameters=x
            )

        if test_features is not None:
            test_features = pd.concat([test_features, t_features], axis=1)
        else:
            test_features = t_features

    test_full_features = pd.merge(test_features, meta_test, left_on=new_features.index, right_on="object_id")
    preds_df = clf.predict(test_full_features)
    preds_df.to_csv('predictions.csv',  header=True, mode='a', index=False)
    

In [ ]:
import pandas as pd

In [ ]:
#test = pd.read_csv("../input/test_set.csv")
meta_test = pd.read_csv("../input/PLAsTiCC-2018/test_set_metadata.csv")
chunks = 5000000
for i_c, test in enumerate(pd.read_csv('../input/PLAsTiCC-2018/test_set.csv', chunksize=chunks, iterator=True)):
    
    test_features = None
    for i in [0,1,2,3,4,5]:
        x = tsfresh.feature_extraction.settings.from_columns(new_features.columns[i*20+1:(i+1)*20+1])
        passband_test = test[test["passband"] == i]
        t_features = tsfresh.extract_features(
                passband_test,
                test_target.loc[passband_train["object_id"].unique()]["target"],
                column_id="object_id", 
                column_sort="mjd",
                column_value="flux",
                kind_to_fc_parameters=x
            )

        if test_features is not None:
            test_features = pd.concat([test_features, t_features], axis=1)
        else:
            test_features = t_features
        
        test_full_features = pd.merge(test_features, meta_test, left_on=new_features.index, right_on="object_id")
        test_full_features.to_csv('test_features.csv',  header=True, mode='a', index=False)

In [ ]:
import tsfresh
import os
import pandas as pd

In [ ]:
new_features = pd.read_csv('../input/new-features/new_features.csv')
meta_train = pd.read_csv('../input/PLAsTiCC-2018/training_set_metadata.csv')
enc = OneHotEncoder()
meta_train['target'] = meta_train['target'].astype("object")
enc.fit(meta_train[["target"]])
target = meta_train[["object_id", "target"]]
target.index = target['object_id']


In [ ]:
x = tsfresh.feature_extraction.settings.from_columns(new_features.columns[i*20+1:(i+1)*20+1])

In [ ]:
new_features.columns[i*20+1:(i+1)*20+1]